In [1]:
%matplotlib inline
import numpy
from matplotlib import pyplot
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [2]:
def rho_green_light(nx,rho_light):
    """Computes "green light" initial condition with shock, and linear distribution behind
    
    Parameters
    ----------
    nx : int
        Number of grid points in x
    rho_light: float
        Density of cars at stoplight
    
    Returns
    -------
    rho: array of floats
        Array with initial values of density
    """
    rho = numpy.arange(nx)*2./nx*rho_light #before stoplight
    rho[(nx-1)/2:]=0
    
    return rho

Numpy.arange creates an array with even steps with length nx
The array is then divided by 2/nx*rho_light
Then you have the density of the traffic set to zero after the (nx-1)/2 point

Object `array` not found.
